# Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers
+ a ballot manifest**
+ a random seed
+ a file of cast vote records
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper cards selected for audit

** The ballot manifest could be only for cards purported to contain the
contests under audit (manifest_type == "STYLE"), or could include cards that might not contain the
contest (manifest_type == "ALL"). These are treated differently. If the sample is to be drawn only from cards that--according to the CVR--contain the contest, and a sampled card turns out not to
contain the contest, that is considered a discrepancy, dealt with using the "phantoms to zombies" approach.
It is assumed that every CVR corresponds to a card in the manifest, but there might
be cards cast in the contest for which there is no corresponding CVR. In that case,
phantom records are created to ensure that the audit is still truly risk-limiting.

Given an independent (i.e., not relying on the voting system) upper bound on the number of cards that contain the contest, if the number of CVRs that contain the contest does not exceed that bound, we can sample from paper purported to contain the contest and use the "zombies" approach (Banuelos & Stark) to deal with missing CVRs. This can greatly increase the efficiency of the audit if the contest is on only a small percentage of the cast cards.

Any sampled phantom card (i.e., a card for which there are no CVRs) is treated as if its CVR is a non-vote (which it is), and as if its MVR was least favorable (a "zombie" producing the greatest doubt in every assertion, separately). Any sampled card for which
there is a CVR is compared to its corresponding CVR. 
If the card turns out not to contain the contest (despite the fact that the CVR says it does), the MVR is treated in the least favorable way for each assertion (i.e., as a zombie rather than as a non-vote).

The tool helps select cards for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited cards.

The current version uses a single sample to audit all contests. It is possible to refine things to target smaller contests.

In [1]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import pandas as pd
import csv

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters, find_margins,\
    find_p_values, find_sample_size, write_audit_parameters
from dominion_tools import \
    prep_dominion_manifest, sample_from_cvr, write_cards_sampled


# Audit parameters.

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `replacement`: whether to sample with replacement. If the sample is drawn with replacement, gamma must also be specified.
* `risk_function`: the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_martingale`. Not all risk functions work with every social choice function. `wald_sprt` applies only to plurality contests.
* `g`: a parameter to hedge against the possibility of observing a maximum overstatement. Require $g \in [0, 1)$ for `kaplan_markov` and `kaplan_wald`
* `N_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.

----

* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `manifest_type`: "STYLE" if the manifest is supposed to list only cards that contain the contests under audit; "ALL" if the manifest contains all cards cast in the election
* `assertion_file`: filename of assertions for IRV contests, in RAIRE format
* `sample_file`: filename for sampled card identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled cards (input)
* `log_file`: filename for audit log (output)

----

* `error_rates`: dict of expected error rates. The keys are
    + `o1_rate`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude in a single round if the audit finds errors
    + `o2_rate`: expected rate of 2-vote overstatements. Recommended value 0.
    + `u1_rate`: expected rate of 1-vote understatements. Recommended value 0.
    + `u2_rate`: expected rate of 2-vote understatements. Recommended value 0.

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are dicts with keys:
        - `risk_limit`: the risk limit for the audit of this contest
        - `cards_cast`: an upper bound on the number of cast cards that contain the contest
        - `choice_function`: `plurality`, `supermajority`, or `IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported; multi-winner super-majority is nonsense)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions

In [2]:
seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = True  # Sampling without replacement isn't implemented
risk_function = "kaplan_martingale"
g=0.1
N_cards = 206036 # total turnout per SF Elections release 9; should limit to VBM turnout

In [3]:
cvr_file = './Data/SFDA2019_PrelimReport9VBMJustDASheets.raire'
manifest_file = './Data/N19 ballot manifest with WH location for RLA Upload.xlsx'
manifest_type = 'STYLE'
sample_file = './Data/sample.csv'
mvr_file = './Data/mvr.json'
log_file = './Data/log.json'

In [4]:
error_rates = {'o1_rate':0.002,      # expect 2 1-vote overstatements per 1000 ballots in the CVR stratum
               'o2_rate':0,          # expect 0 2-vote overstatements
               'u1_rate':0,          # expect 0 1-vote understatements
               'u2_rate':0}          # expect 0 2-vote understatements

In [5]:
# contests to audit. Contest 339 is the DA race
contests = {'339':{'risk_limit':0.05,
                     'choice_function':'IRV',
                     'n_winners':1,
                     'candidates':['15','16','17','18','45'],
                     'reported_winners' : ['15'],
                     'assertion_file' : './Data/SF2019Nov8Assertions.json'
                    }
           }

Example of other social choice functions:

> contests =  {'city_council':{'risk_limit':0.05,
                     'choice_function':'plurality',
                     'n_winners':3,
                     'candidates':['Doug','Emily','Frank','Gail','Harry'],
                     'reported_winners' : ['Doug', 'Emily', 'Frank']
                    },
            'measure_1':{'risk_limit':0.05,
                     'choice_function':'supermajority',
                     'share_to_win':2/3,
                     'n_winners':1,
                     'candidates':['yes','no'],
                     'reported_winners' : ['yes']
                    }                  
           }

## Find audit parameters and conduct audit

* Import contest data
    - ballot manifest
    - cast vote records (CVRs)

* Set up data for the phantom/zombie treatment of missing CVRs and cards not listed in the manifest
    - create empty CVRs and MVRs for unaccounted-for cards that could contain the contests

* For each contest:
    - find claimed outcome by applying SCF to CVRs
    - complain if claimed outcome disagrees with reported outcome
    - construct assertions that imply contest outcome is correct
    - for each assertion:
        + find generalized diluted margin
        
* Find initial (incremental) sample size from smallest diluted margin, for the sampling plan
    - Complain if expected error rates imply any assertion is incorrect

* For each assertion:
    - Initialize discrepancy counts to zero (o1, o2, u1, u2)
    - Initialize measured risk to 1

* While measured risk for any assertion exceeds its risk limit:
    - expand sample by estimated increment
        + identify sampled cards in manifest
        + update the log file with incremental sample
    - import audit results when cards have been audited
    - for each assertion:
        + for each sampled card:
            - increment discrepancy count for the assertion
        + find measured risk
    - update log file with new measured risks
    - if any measured risk exceeds its risk limit:
        + estimate incremental sample required to complete the audit

In [6]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c]['choice_function'] == 'IRV':
        with open(contests[c]['assertion_file'], 'r') as f:
            contests[c]['assertion_json'] = json.load(f)['audits'][0]['assertions']

In [7]:
# construct the dict of dicts of assertions for each contest
all_assertions = Assertion.make_all_assertions(contests)

In [8]:
all_assertions

{'339': {'18 v 17 elim 15 16 45': <assertion_audit_utils.Assertion at 0x106fa5518>,
  '17 v 16 elim 15 18 45': <assertion_audit_utils.Assertion at 0x106fa52b0>,
  '15 v 18 elim 16 17 45': <assertion_audit_utils.Assertion at 0x106fa5400>,
  '18 v 16 elim 15 17 45': <assertion_audit_utils.Assertion at 0x106fa5358>,
  '17 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x106fa5320>,
  '15 v 17 elim 16 45': <assertion_audit_utils.Assertion at 0x106fa5080>,
  '15 v 17 elim 16 18 45': <assertion_audit_utils.Assertion at 0x106fa51d0>,
  '18 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x106fa5128>,
  '15 v 16 elim 17 45': <assertion_audit_utils.Assertion at 0x106fa50f0>,
  '15 v 16 elim 17 18 45': <assertion_audit_utils.Assertion at 0x106fa5048>,
  '15 v 16 elim 18 45': <assertion_audit_utils.Assertion at 0x106fa5780>,
  '15 v 16 elim 45': <assertion_audit_utils.Assertion at 0x10f976fd0>,
  '15 v 45': <assertion_audit_utils.Assertion at 0x10f976f28>}}

## Read the ballot manifest

In [9]:
# special for SF/Dominion manifest format
manifest = pd.read_excel(manifest_file)

## Read the CVRs 

In [10]:
cvr_input = []
with open(cvr_file) as f:
    cvr_reader = csv.reader(f, delimiter=',', quotechar='"')
    for row in cvr_reader:
        cvr_input.append(row)

print("Read {} rows".format(len(cvr_input)))

Read 146606 rows


In [11]:
# Import CVRs
cvr_list = CVR.from_raire(cvr_input)
print("After merging, there are CVRs for {} cards".format(len(cvr_list)))

After merging, there are CVRs for 146604 cards


In [12]:
# turn RAIRE-style identifiers into SF-style by substituting "-" for "_"
for c in cvr_list:
    c.set_id(str(c.id).replace("_","-"))

In [13]:
for i in range(10):
    print(cvr_list[i].id)

99813-1-1
99813-1-3
99813-1-6
99813-1-8
99813-1-9
99813-1-11
99813-1-13
99813-1-16
99813-1-17
99813-1-19


In [14]:
# Check that there is a CVR for every card cast in the contest. If not, add phantoms.

n_cvrs = len(cvr_list)
manifest, manifest_cards, phantom_cards = prep_dominion_manifest(manifest, N_cards, n_cvrs)

manifest

/Users/stark/Dropbox/Letters/Ms/Vote/Assertion19/Code/dominion_tools.py:39: UserWarning: The CVR list does not account for every card cast in the contest; adding a phantom batch to the manifest
  warnings.warn('The CVR list does not account for every card cast in the contest; adding a phantom batch to the manifest')


,Tray #,Tabulator Number,Batch Number,Total Ballots,VBMCart.Cart number,cum_cards
0,1,99808,78,116,3.0,116
1,1,99808,77,115,3.0,231
2,1,99808,79,120,3.0,351
3,1,99808,81,76,3.0,427
4,1,99808,80,116,3.0,543
5,2,99804,7,129,2.0,672
6,2,99804,9,100,2.0,772
7,2,99804,10,97,2.0,869
8,2,99804,8,104,2.0,973
9,2,99804,6,118,2.0,1091


In [15]:
# Create CVRs and MVRs for phantom cards
# If the sample draws a phantom card, these CVRs will be used in the comparison.
# phantom MVRs should be treated as zeros by the Assorter for every contest
phantom_vrs = []
for i in range(phantom_cards):
    phantom_vrs.append(CVR(id='phantom-1-'+str(i+1), votes={}, phantom = True))  # matches expected RAIRE id for parsing later
    
cvr_list = cvr_list + phantom_vrs

print("Created {} phantom records".format(len(phantom_vrs)))

Created 59432 phantom records


In [16]:
manifest_cards

321110

In [17]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
min_margin = find_margins(contests, all_assertions, cvr_list)

print("minimum assorter margin {}".format(min_margin))
for c in contests:
    print("margins in contest {}".format(c))
    for a, m in contests[c]['margins'].items():
        print(a, m)

minimum assorter margin 0.014114038323399747
margins in contest 339
18 v 17 elim 15 16 45 0.032639927003047964
17 v 16 elim 15 18 45 0.014114038323399747
15 v 18 elim 16 17 45 0.020622609641033574
18 v 16 elim 15 17 45 0.059047933370867334
17 v 16 elim 15 45 0.04129375448950667
15 v 17 elim 16 45 0.05744627152536452
15 v 17 elim 16 18 45 0.07800093187598289
18 v 16 elim 15 45 0.1058455803840106
15 v 16 elim 17 45 0.09643945718224001
15 v 16 elim 17 18 45 0.09719660641829586
15 v 16 elim 18 45 0.11862975402356879
15 v 16 elim 45 0.11121357432681678
15 v 45 0.21039527072938702


In [18]:
check_audit_parameters(risk_function, g, error_rates, contests)

In [19]:
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_cards, n_cvrs, \
                       manifest_cards, phantom_cards, error_rates, contests)

## Set up for sampling

## Find initial sample size

In [ ]:
# find initial sample size
ss_fn = lambda m, r: TestNonnegMean.kaplan_martingale_sample_size(N_cards, m,\
                    overstatement_rate = 0.001, alpha=r)
sample_size = find_sample_size(contests, all_assertions, sample_size_function = ss_fn)
sample_size

/Users/stark/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


## Draw the first sample

In [ ]:
# draw the initial sample
prng = SHA256(seed)
sample = sample_by_index(N_cards, sample_size, prng=prng) # 1-indexed
n_phantom_sample = np.sum([cvr_list[i].phantom for i in sample])
print("The sample includes {} phantom cards.".format(n_phantom_sample))

In [ ]:
cvr_sample_lookup, cvr_sample, mvr_phantoms_sample = sample_from_cvr(cvr_list, manifest, sample)

In [ ]:
# write the sample
write_cards_sampled(sample_file, cvr_sample_lookup, print_phantoms=False)

## Read the audited sample data

In [ ]:
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

In [ ]:
# add MVRs for phantoms
mvr_sample = mvr_sample + mvr_phantoms_sample

## Find measured risks for all assertions

In [ ]:
## **FOR DEVELOPMENT TESTING ONLY** DELETE THIS CELL FOR PRODUCTION
mvr_sample = cvr_sample
np.random.shuffle(mvr_sample)
for i in range(len(sample)):
    mvr_sample[i].phantom = False
    cvr_sample[i].phantom = False

In [ ]:
# order the MVRs and CVRs
mvr_sample.sort(key= lambda x: x.id)
cvr_sample.sort(key= lambda x: x.id)

# check that the IDs of sampled mvrs and cvrs match
assert len(cvr_sample) == len(mvr_sample),\
    "Number of cvrs ({}) and mvrs ({}) doesn't match".format(len(cvr_sample), len(mvr_sample))
for i in range(len(cvr_sample)):
    assert mvr_sample[i].id == cvr_sample[i].id, \
    "Mismatch between id of cvr ({}) and mvr ({})".format(cvr_sample[i].id, mvr_sample[i].id)

In [ ]:
p_max = find_p_values(contests, all_assertions, mvr_sample, cvr_sample, manifest_type, \
                      risk_function= lambda x: TestNonnegMean.kaplan_martingale(x, N_cards)[0])
print("maximum assertion p-value {}".format(p_max))

done = True
for c in contests:
    print("p-values for assertions in contest {}".format(c))
    cpmax = 0
    for a in all_assertions[c]:
        p = all_assertions[c][a].p_value
        cpmax = np.max([cpmax,p])
        print(a, p)
    if cpmax <= contests[c]['risk_limit']:
        print("contest {} AUDIT COMPLETE at risk limit {}. Attained risk {}".format(\
            c, contests[c]['risk_limit'], cpmax))
    else:
        done = False
        print("contest {} audit INCOMPLETE at risk limit {}. Attained risk {}".format(\
            c, contests[c]['risk_limit'], cpmax))

In [ ]:
# Identify assertions not yet confirmed
if not done:
    pass

In [ ]:
# Log the status of the audit 

# Escalation: how many more cards should be audited?

This tool estimates how many more cards will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* cards already sampled
* assumption that we will continue to see errors at the same rate observed in the sample

In [ ]:
sample_sizes_new = {}

# TBD


In [ ]:
# augment the sample
# reset the seed
prng = SHA256(seed)
old_sample = sample
sample = sample_by_index(N_cards, sample_size, prng=prng)
incremental_sample = np.sort(list(set(sample) - set(old_sample)))
